# Installing necessary libraries

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.9 MB/s eta 0:00:00


# Importing necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import csv

import transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizer

# Import our textual data

### Importing Dataset & Indexing the columns to identify text in each column

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# Load the train dataset
path1 = "/content/drive/MyDrive/Colab Notebooks/Hamid_DL_Assignment/Hamid2/train.csv"
with open(path1, 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\n')
    df_train = pd.DataFrame([row[0].split(',') for row in reader])

# Load the test dataset
path2 = "/content/drive/MyDrive/Colab Notebooks/Hamid_DL_Assignment/Hamid2/test.csv"
with open(path2, 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\n')
    df_test = pd.DataFrame([row[0].split(',') for row in reader])

# Remove the first row of the DataFrames
df_train = df_train.drop(0)
df_test = df_test.drop(0)

#defining an initial label for test dataset just for the sake of size consistency 
# insert the label column at index 1 (just to make it similar to df_train to avoid error)
df_test.insert(1,'label', 0) 

# print for information (for checking)
df_train.head(2)


Mounted at /content/drive


,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
1,0,1,"""Wizards of the Lost Kingdom is a movie about ...",plot,acting,and generally everything about this movie is ...,it's so bad that it's funny. You will keep wa...,and,like the other reviewer of the movie said,it's so bad it's good.,...,None,None,None,None,None,None,None,None,None,None
2,1,1,Brilliant adaptation of the largely interior m...,Stephen Dedalus,and Molly Bloom by Joseph Strick in recreatin...,1904 - Bloomsday - a day to be celebrated - d...,screenwriter Haines,as well as casting director and cinematograph...,The Tin Drum filmed by Volker Schlöndorff (19...,there are also recognizable characters,...,None,None,None,None,None,None,None,None,None,None


### Merge text data to reach a new unique column named "merged_text"


In [ ]:

#merge text data and make a new column "merged_text"
df_train['merged_text'] = df_train.iloc[:, 2:].apply(lambda x: ' '.join(x.astype(str)), axis=1)
df_test['merged_text']  = df_test.iloc[:, 2:].apply(lambda x: ' '.join(x.astype(str)), axis=1)

# Extra printing for checking
print("shape", len(df_train))
df_train['merged_text'].head(2)


shape 25000


1    "Wizards of the Lost Kingdom is a movie about ...
2    Brilliant adaptation of the largely interior m...
Name: merged_text, dtype: object

### Making a summerized TEXT dataframe including "id", "label" and "merged text"


In [ ]:
# making a new column including "id", "label" and "merged text"
df_train_text = pd.DataFrame(df_train.iloc[:, 0:2])
df_train_text['merged_text'] = df_train['merged_text']
df_train_text.columns = ['id', 'label', 'merged_text']

print("train, \n", df_train_text.head())

df_test_text = pd.DataFrame(df_test.iloc[:, 0:2])
df_test_text['merged_text'] = df_test['merged_text']
df_test_text.columns = ['id', 'label', 'merged_text']

# Extra priting
print("test, \n", df_test_text.head())
##print("test, \n", df_test_text['merged_text'].head())
##print("test label, \n", df_test_text['label'].head())

# Remove the "NONE" sequence'
for i in range(len(df_train_text)):
    df_train_text.iloc[i, 2] = df_train_text.iloc[i, 2].replace("None", "")
    df_test_text.iloc[i, 1] = df_test_text.iloc[i, 2].replace("None", "")

#print("test, \n", df_test_text['merged_text'].head())

data = df_train_text[['merged_text', 'label']]
data.columns = ['review', 'class']
data2 = df_test_text[['merged_text', 'label']]
data2.columns = ['review', 'class']

train, 
   id label                                        merged_text
1  0     1  "Wizards of the Lost Kingdom is a movie about ...
2  1     1  Brilliant adaptation of the largely interior m...
3  2     1  Typical De Palma movie made with lot's of styl...
4  3     1  This is a fantastic series first and foremost....
5  4     1  My only minor quibble with the film I grew up ...
test, 
       id  label                                        merged_text
1  25000      0  Stanwyck at her villainous best  Robinson her ...
2  25001      0  I'm surprised by some of the comments on this ...
3  25002      0  I remember a time when the only thing that did...
4  25003      0  What can I say? The little kid inside has alwa...
5  25004      0  This is my favorite horror film  a close 2nd t...


In [ ]:
# Extra printing
print("Length of df_train_text:", len(df_test_text['merged_text']))
data.head(3)
data2.head(3)

Length of df_train_text: 25000


,review,class
1,Stanwyck at her villainous best Robinson her ...,Stanwyck at her villainous best Robinson her ...
2,I'm surprised by some of the comments on this ...,I'm surprised by some of the comments on this ...
3,I remember a time when the only thing that did...,I remember a time when the only thing that did...


# Preprocessing reviews

**Here, I'm using a text preporcessing library found in Github**

In [ ]:
!git clone https://github.com/behdad13/preprocessing_text_ben.git/

Cloning into 'preprocessing_text_ben'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 27 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (27/27), 6.76 KiB | 1.13 MiB/s, done.


In [ ]:
cd preprocessing_text_ben

/content/preprocessing_text_ben


In [ ]:
import preprocessing_text_ben as pp

In [ ]:
cd-

/content


## one-shot data cleaning

In [ ]:
def get_clean(x):
    
    # Replace any backslashes with empty string and underscores with spaces
    x = str(x).replace('\\', '').replace('_', ' ')
    
    # Convert the string to lowercase
    x = str(x).lower()
    
    # Expand contractions like "don't" to "do not"
    x = pp.cont_to_exp(x)
    
    # Remove any email addresses from the string
    x = pp.remove_emails(x)
    
    # Remove any URLs from the string
    x = pp.remove_urls(x)
    
    # Remove any HTML tags from the string
    x = pp.remove_html_tags(x)
    
    # Remove any re-review tags (RT) from the string
    x = pp.remove_rt(x)
    
    # Remove any accented characters from the string
    x = pp.remove_accented_chars(x)
    
    # Remove any special characters from the string
    x = pp.remove_special_chars(x)
    
    # Replace any repeated characters with a single instance
    x = re.sub("(.)\1{2,}", "\1", x)
    
    # Return the cleaned string
    return x

In [ ]:
#data = data[['review', 'class']]
data['class'].value_counts()
data['review'].head(2)

1    "Wizards of the Lost Kingdom is a movie about ...
2    Brilliant adaptation of the largely interior m...
Name: review, dtype: object

In [ ]:
data.review = data.review.apply(lambda x: get_clean(x))
data2.review = data2.review.apply(lambda x: get_clean(x))

data.head(5)

<ipython-input-16-ec36be05a38f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.review = data.review.apply(lambda x: get_clean(x))
<ipython-input-16-ec36be05a38f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2.review = data2.review.apply(lambda x: get_clean(x))


,review,class
1,wizards of the lost kingdom is a movie about a...,1
2,brilliant adaptation of the largely interior m...,1
3,typical de palma movie made with lots of style...,1
4,this is a fantastic series first and foremost ...,1
5,my only minor quibble with the film i grew up ...,1


### Shuffle twice the train dataset before splitting to train/val/test set

In [ ]:
data_shuffled = data.sample(frac=1)
data_shuffled = data_shuffled.sample(frac=1)
data = data_shuffled
data.head(3)

,review,class
15919,this movie is a mess but at least it is not pr...,0
15601,my main comment on this movie is how zwick was...,0
6102,the reviewer who called this movie a bust has ...,1


In [ ]:
# EXTRA Printing
print("first review:", data.review[1])

##data = data[['review', 'class']]
##data['class'].value_counts()
##print("type", type(data.review[1]))
##print("second review:", data.review[2])

data.head(3)

first review: wizards of the lost kingdom is a movie about a young prince simon who is banished from his kingdom due to his father the king being killed by the cliche evil adviser this movies about simons adventures the special effects plot acting and generally everything about this movie is bad however it is so bad that it is funny you will keep watching this movie simply because it is so bad it is funny and like the other reviewer of the movie said it is so bad it is good


,review,class
15919,this movie is a mess but at least it is not pr...,0
15601,my main comment on this movie is how zwick was...,0
6102,the reviewer who called this movie a bust has ...,1


 **Splitting the train dataset into train/val/test**

In [ ]:
train_ratio = 0.80
valid_ratio = 0.20
data_train = data[:int(train_ratio*len(data))]
data_valid = data[int(train_ratio*len(data)):int((valid_ratio+train_ratio)*len(data))]


In [ ]:
#################################### END of Preprocessing #########################################3

# Model tokenizer

**Load the pre-trained RoBERTa-large model and tokenizer**


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load the pre-trained RoBERTa-Large model and tokenizer
model_name = 'roberta-large'
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = RobertaTokenizer.from_pretrained(model_name)

##############################################
#from transformers import AutoTokenizer
#tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classi

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, max_length=self.max_length, padding='max_length', return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0],
            'labels': torch.tensor(int(label), dtype=torch.long)
        }

In [ ]:
train_texts, train_labels = list(data_train.review), list(data_train['class'])
val_texts, val_labels = list(data_valid.review), list(data_valid['class'])

In [ ]:
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer)

# Training model

In [ ]:
# Set GPU as default device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

**Setting the traning arguments**




In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=20,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    learning_rate=2e-5
)

**Define the trainer**


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.211200,0.220514,0.916200,0.916542,0.916200,0.916165
2,0.159300,0.235178,0.917400,0.917505,0.917400,0.917385
3,0.079900,0.337836,0.923200,0.923920,0.923200,0.923144
4,0.151900,0.377423,0.923600,0.923683,0.923600,0.923588
5,0.044900,0.448113,0.925200,0.925215,0.925200,0.925195


TrainOutput(global_step=6250, training_loss=0.17463989980831743, metrics={'train_runtime': 8409.3458, 'train_samples_per_second': 11.892, 'train_steps_per_second': 0.743, 'total_flos': 2.32982840832e+16, 'train_loss': 0.17463989980831743, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.4481127858161926,
 'eval_accuracy': 0.9252,
 'eval_precision': 0.9252149395932611,
 'eval_recall': 0.9252,
 'eval_f1': 0.9251954748188129,
 'eval_runtime': 114.8542,
 'eval_samples_per_second': 43.533,
 'eval_steps_per_second': 2.177,
 'epoch': 5.0}

# Test the model

**Predicting on the test dataset**

In [ ]:
import torch

# function to predict on a batch of texts
def predict_batch(texts, model, tokenizer, device, batch_size=32):
    predicted_classes = []
    probabilities = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt').to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            batch_probabilities = torch.softmax(logits, dim=-1)
            batch_predicted_classes = torch.argmax(batch_probabilities, dim=-1).tolist()
        predicted_classes.extend(batch_predicted_classes)
        probabilities.extend(batch_probabilities.cpu().numpy().tolist())
    return predicted_classes, probabilities
############################################
# taking all test data
test_texts = data2.review.tolist()
predicted_classes, probabilities = predict_batch(test_texts, model, tokenizer, device, batch_size=32)

In [ ]:
# create a DataFrame with predicted classes and probabilities
print(len(predicted_classes))
results_df = pd.DataFrame({'id': range(25000, 50000),
                           'label': predicted_classes})
# save the results to a CSV file
results_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Hamid_DL_Assignment/Hamid2/results_ROBERTA Large_Epoch20.csv', index=False)

results_df


25000


,id,label
0,25000,1
1,25001,1
2,25002,1
3,25003,1
4,25004,1
...,...,...
24995,49995,1
24996,49996,1
24997,49997,0
24998,49998,1


# Conclusion on my contribution and findings


1- I investigated multiple pre-trained models with a vast corpus and fine-tuned them for the sentiment analysis task (binary classification) of the dataset we were given. I tried BERT, DISTIL-BERT, ROBERTA, and ROBERTA LARGE. By using ROBERTA LARGE I achieved the highest accuracy for both model and also test set.

2- I also tried to work on tuning the hyperparameters. I played with learning rate, #epochs, and batch size to see how they affect the model performance! I tested multiple combinations of them and finally, It turned out that an epoch more than 5 or 6, is no longer much helpful. This was the same when increasing batch size after 16. 

3- Finally considering different pre-trained models and different combinations of hyperparameters tuning, I achieved close to 20 different models out of which I submitted 9 of them which resulted in achieving accuracy on the test set in the range of 87%, up to 95%.

